In [30]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tabular-playground-series-mar-2021/sample_submission.csv
/kaggle/input/tabular-playground-series-mar-2021/train.csv
/kaggle/input/tabular-playground-series-mar-2021/test.csv


In [31]:
import matplotlib.pyplot as plt
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

# Import Datsets


In [32]:
train = pd.read_csv('../input/tabular-playground-series-mar-2021/train.csv')
test = pd.read_csv('../input/tabular-playground-series-mar-2021/test.csv')

In [33]:
print(f'Training Data\n-----------\n{train.shape}\n {train.head()}\n\n')
print(f'Test Data\n-----------\n{test.shape}\n {test.head()}')

Training Data
-----------
(300000, 32)
    id cat0 cat1 cat2 cat3 cat4 cat5 cat6 cat7 cat8  ...     cont2     cont3  \
0   0    A    I    A    B    B   BI    A    S    Q  ...  0.759439  0.795549   
1   1    A    I    A    A    E   BI    K    W   AD  ...  0.386385  0.541366   
2   2    A    K    A    A    E   BI    A    E   BM  ...  0.343255  0.616352   
3   3    A    K    A    C    E   BI    A    Y   AD  ...  0.831147  0.807807   
4   4    A    I    G    B    E   BI    C    G    Q  ...  0.338818  0.277308   

      cont4     cont5     cont6     cont7     cont8     cont9    cont10 target  
0  0.681917  0.621672  0.592184  0.791921  0.815254  0.965006  0.665915      0  
1  0.388982  0.357778  0.600044  0.408701  0.399353  0.927406  0.493729      0  
2  0.793687  0.552877  0.352113  0.388835  0.412303  0.292696  0.549452      0  
3  0.800032  0.619147  0.221789  0.897617  0.633669  0.760318  0.934242      0  
4  0.610578  0.128291  0.578764  0.279167  0.351103  0.357084  0.328960      1  

# Feature Engineering

# * Encoding Categorical Features

In [34]:
train.drop('id',axis=1,inplace=True)
test_id = test['id']
test.drop('id',axis=1,inplace=True)


In [35]:
categorical = list(train.select_dtypes('object').columns)
categorical

['cat0',
 'cat1',
 'cat2',
 'cat3',
 'cat4',
 'cat5',
 'cat6',
 'cat7',
 'cat8',
 'cat9',
 'cat10',
 'cat11',
 'cat12',
 'cat13',
 'cat14',
 'cat15',
 'cat16',
 'cat17',
 'cat18']

In [36]:
X = train.iloc[:,:-1].copy()
y = train['target'].copy()
col_transform = ColumnTransformer(transformers=[['ohe',OneHotEncoder(),categorical]],remainder='passthrough')
X = col_transform.fit_transform(X).copy()

In [37]:
print(X.shape)
print(y.shape)

(300000, 634)
(300000,)


In [8]:
#taking care of missing data
imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
imputer.fit(temp)
temp = imputer.transform(temp)

In [9]:
train = temp[:train_len,1:] #excluding ID
test = temp[train_len:,1:-1] #excluding ID and Target columns

In [10]:
print(f'Training Data\n-----------\n {train.shape}\n\n {train}\n\n')
print(f'Test Data\n-----------\n {test.shape}\n\n {test}')

Training Data
-----------
 (300000, 624)

 [[0.62985809 0.85534905 0.7594387  ... 1.         0.         0.        ]
 [0.3707271  0.32892942 0.38638484 ... 1.         0.         0.        ]
 [0.5022717  0.32274884 0.34325474 ... 1.         0.         0.        ]
 ...
 [0.48788237 0.52234741 0.40703655 ... 0.         0.         1.        ]
 [0.33190029 0.81289071 0.80804501 ... 0.         0.         0.        ]
 [0.8226001  0.81973454 0.77545056 ... 1.         0.         0.        ]]


Test Data
-----------
 (200000, 623)

 [[0.70811972 0.73569008 0.57836625 ... 1.         1.         0.        ]
 [0.61163698 0.31370333 0.92888511 ... 1.         1.         0.        ]
 [0.45628881 0.44820121 0.42487624 ... 1.         1.         0.        ]
 ...
 [0.46306686 0.8123842  0.80334774 ... 1.         1.         0.        ]
 [0.51967073 0.81128173 0.82063505 ... 1.         1.         0.        ]
 [0.49832813 0.33616841 0.27048292 ... 0.         1.         0.        ]]



* Spilitting the data


In [11]:
X_train, X_test, y_train, y_test = train_test_split(train[:,:-1], train[:,-1], test_size = 0.2, random_state = 1)

# Scaling

In [12]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# PCA

In [13]:
pca = PCA(n_components = 0.99)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

# Model selection 
# * Random Forest Classifier

In [ ]:
classifier = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 0)
classifier.fit(X_train, y_train)

# * LGBM

In [16]:
classifier = lgb.LGBMClassifier()
classifier.fit(X_train, y_train)

LGBMClassifier()

LGBMClassifier()

In [19]:
# Model Evaluation
print(classifier)
y_pred = classifier.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
cr = classification_report(y_test,y_pred)
print(f'Confusion Matrix:\n {cm}\n')
print(f'Classification Report:\n {cr}\n')
print(f'Accuracy Score = {accuracy_score(y_test, y_pred) *100 :.2f}%')

LGBMClassifier()
Confusion Matrix:
 [[55077   492]
 [ 1789  2642]]

Classification Report:
               precision    recall  f1-score   support

         0.0       0.97      0.99      0.98     55569
         1.0       0.84      0.60      0.70      4431

    accuracy                           0.96     60000
   macro avg       0.91      0.79      0.84     60000
weighted avg       0.96      0.96      0.96     60000


Accuracy Score = 96.20%


In [23]:
#fittimg the whole train data set into the model
train = sc.fit_transform(train)
train = pca.fit_transform(train)
classifier.fit(train[:,:-1], train[:,-1])

ValueError: Unknown label type: 'continuous'

In [173]:
#predciting the given test set
predictions = classifier.predict(test)

# File Submission

In [174]:
submission = pd.DataFrame({'id': test_id,'target':predictions}) 
submission.to_csv('submission.csv',index=False)

In [175]:
submission

,id,target
0,5,0.0
1,6,0.0
2,8,0.0
3,9,0.0
4,11,0.0
...,...,...
199995,499983,0.0
199996,499984,0.0
199997,499987,0.0
199998,499994,0.0
